# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
path = os.path.join(os.getcwd(),"temperaturedata.csv")
data = pd.read_csv(path)
data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date
0,Druzhba,52.0428,33.9453,68.04,94,47,7.18,UA,1624405444
1,Dikson,73.5069,80.5464,31.86,86,100,15.93,RU,1624405444
2,Mâncio Lima,-7.6142,-72.8958,75.33,76,81,2.37,BR,1624405444
3,Ivankiv,50.9386,29.8943,73.60,92,36,4.54,UA,1624405445
4,Hobart,-42.8794,147.3294,46.11,91,0,3.00,AU,1624405218


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#configure gmaps
gmaps.configure(api_key=g_key)
# store locations
locations = data[["Lat","Lng"]].astype(float)
# store weights
humidity_percent = data["Humidity"].astype(float)

In [4]:
# Create a humidity heatmap layer
fig = gmaps.figure()

humidity_layer = gmaps.heatmap_layer(locations, weights=humidity_percent, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(humidity_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
data2 = data
# dropping rows with clouds
data2 = data2.loc[(data2["Cloudiness"] == 0)]
# Dropping rows with strong winds
data2 = data2.loc[(data2["Windspeed"] < 10)]
# Dropping rows with very cold max temps
data2 = data2.loc[(data2["Max Temp"] < 80)]
data2 = data2.loc[(data2["Max Temp"] > 70)]
# displaying data
len(data2["City"])

14

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
hotel_df = data2
hotel_df["Hotel Name"] = ""
target_search = "Hotel"
target_type = "Lodging"
target_radius = 5000

params = {
    'keyword':target_search,
    'radius':target_radius,
    'type':target_type,
    'key':g_key
}

for index, row in hotel_df.iterrows():
    latitude = row["Lat"]
    longitude = row["Lng"]

    params["location"] = f"{latitude},{longitude}"

    endpoint = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(endpoint, params=params).json()
    # print(json.dumps(response, indent=4, sort_keys=True))

    hotels = response["results"]
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels[0]["name"]
        print(f"Hotel found in {index}")
    except (KeyError, IndexError):
        print(f"No Hotels found in {index}")


Hotel found in 65
Hotel found in 79
Hotel found in 142
Hotel found in 261
No Hotels found in 300
Hotel found in 347
Hotel found in 381
Hotel found in 382
No Hotels found in 396
Hotel found in 400
Hotel found in 407
Hotel found in 441
Hotel found in 465
No Hotels found in 532


In [13]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date,Hotel Name
65,Porto-Vecchio,41.5910,9.2795,72.93,94,0,3.44,FR,1624405463,Hôtel Le Goéland - Casa Santini
79,Lixouri,38.2019,20.4314,77.99,51,0,7.47,GR,1624405467,Thalassa Boutique Hotel Kefalonia
142,Ardakān,32.3100,54.0175,77.38,11,0,4.63,IR,1624405486,Kheshtomah Ecolodge
261,Dakar,14.6937,-17.4441,78.93,83,0,4.61,SN,1624405313,"Radisson Blu Hotel, Dakar Sea Plaza"
300,Baghlan,36.1307,68.7083,73.71,21,0,3.69,AF,1624405535,
347,Akdepe,42.0551,59.3788,71.69,26,0,9.73,TM,1624405549,Мой дом
381,Palic,46.1023,19.7635,73.89,61,0,5.75,RS,1624405560,Hotel Garni Palić Resort
382,Dārāb,28.7519,54.5444,77.58,8,0,8.41,IR,1624405561,Naghshe shapoor Hotel
396,Faya,18.3851,42.4509,71.71,13,0,8.57,SA,1624405565,
400,Hefei,31.8639,117.2808,78.94,83,0,2.24,CN,1624405566,"Shangri-La Hotel, Hefei"


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
markers = gmaps.marker_layer(locations)

In [15]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)



# Display figure
fig = gmaps.figure()
fig.add_layer(hotel_layer)
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))